In [ ]:
import os
import time
import cv2
import depthai as dai
import numpy as np
import blobconverter
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import display, Image

from utils.priorbox import PriorBox
from utils.utils import draw

confidence_thresh = 0.6
iou_thresh=0.3
keep_top_k = 750
NN_WIDTH, NN_HEIGHT = 160, 120
VIDEO_WIDTH, VIDEO_HEIGHT = 640, 480

pipeline = dai.Pipeline()
pipeline.setOpenVINOVersion(version=dai.OpenVINO.VERSION_2021_4)

detection_nn = pipeline.create(dai.node.NeuralNetwork)
detection_nn.setBlobPath(blobconverter.from_zoo(name="face_detection_yunet_160x120", zoo_type="depthai", shaves=6))
detection_nn.setNumPoolFrames(4)
detection_nn.input.setBlocking(False)
detection_nn.setNumInferenceThreads(2)

cam = pipeline.create(dai.node.ColorCamera)
cam.setPreviewSize(VIDEO_WIDTH, VIDEO_HEIGHT)
cam.setInterleaved(False)
cam.setFps(40)
cam.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)

manip = pipeline.create(dai.node.ImageManip)
manip.initialConfig.setResize(NN_WIDTH, NN_HEIGHT)
manip.initialConfig.setFrameType(dai.RawImgFrame.Type.BGR888p)
manip.inputConfig.setWaitForMessage(False)

xout_cam = pipeline.create(dai.node.XLinkOut)
xout_cam.setStreamName("cam")

xout_nn = pipeline.create(dai.node.XLinkOut)
xout_nn.setStreamName("nn")

cam.preview.link(manip.inputImage)
cam.preview.link(xout_cam.input)
manip.out.link(detection_nn.input)
detection_nn.out.link(xout_nn.input)


with dai.Device(pipeline) as device:
    q_cam = device.getOutputQueue("cam", 4, blocking=False)  # type: ignore
    q_nn = device.getOutputQueue(name="nn", maxSize=4, blocking=False)  # type: ignore
    start_time = time.time()
    counter = 0
    fps = 0.0
    layer_info_printed = False
    display_handle=display(None, display_id=True)
    while True:
        in_frame = q_cam.get()
        in_nn = q_nn.get()
        frame = in_frame.getCvFrame()
        conf = np.array(in_nn.getLayerFp16("conf")).reshape((1076, 2))
        iou = np.array(in_nn.getLayerFp16("iou")).reshape((1076, 1))
        loc = np.array(in_nn.getLayerFp16("loc")).reshape((1076, 14))
        pb = PriorBox(
            input_shape=(NN_WIDTH, NN_HEIGHT),
            output_shape=(frame.shape[1], frame.shape[0]),
        )
        dets = pb.decode(loc, conf, iou, confidence_thresh)
        if dets.shape[0] > 0:
            bboxes = dets[:, 0:4]
            scores = dets[:, -1]
            keep_idx = cv2.dnn.NMSBoxes(
                bboxes=bboxes.tolist(),
                scores=scores.tolist(),
                score_threshold=confidence_thresh,
                nms_threshold=iou_thresh,
                eta=1,
                top_k=keep_top_k,
            )  # returns [box_num, class_num]
            keep_idx = np.squeeze(keep_idx)  # [box_num, class_num] -> [box_num]
            dets = dets[keep_idx]
        if dets.shape[0] > 0:
            if dets.ndim == 1:
                dets = np.expand_dims(dets, 0)
            img_res = draw(
                img=frame,
                bboxes=dets[:, :4],
                landmarks=np.reshape(dets[:, 4:14], (-1, 5, 2)),
                scores=dets[:, -1],
            )
        color_black, color_white = (0, 0, 0), (255, 255, 255)
        label_fps = "Fps: {:.2f}".format(fps)
        (w1, h1), _ = cv2.getTextSize(label_fps, cv2.FONT_HERSHEY_TRIPLEX, 0.4, 1)
        cv2.rectangle(
            frame,
            (0, frame.shape[0] - h1 - 6),
            (w1 + 2, frame.shape[0]),
            color_white,
            -1,
        )
        cv2.putText(
            frame,
            label_fps,
            (2, frame.shape[0] - 4),
            cv2.FONT_HERSHEY_TRIPLEX,
            0.4,
            color_black,
        )
        resizedFrame = cv2.resize(frame, (int(columns/2), int(rows/2)))
        _, jpg = cv2.imencode('.jpeg', resizedFrame)
        display_handle.update(Image(data=jpg.tobytes()))
        counter += 1
        if (time.time() - start_time) > 1:
            fps = counter / (time.time() - start_time)
            counter = 0
            start_time = time.time()
        if cv2.waitKey(1) == ord("q"):
            break
